In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Layer
from tensorflow.keras import layers, initializers
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('NYCDataset3years/NYCTraffic_2006_2008.csv')
df.info()
df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81035 entries, 0 to 81034
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Yr         81035 non-null  int64 
 1   M          81035 non-null  int64 
 2   D          81035 non-null  int64 
 3   HH         81035 non-null  int64 
 4   MM         81035 non-null  int64 
 5   Vol        81035 non-null  int64 
 6   SegmentID  81035 non-null  int64 
 7   Direction  81035 non-null  object
dtypes: int64(7), object(1)
memory usage: 4.9+ MB


,Yr,M,D,HH,MM,Vol,SegmentID,Direction
0,2008,9,8,9,0,320,17120,SB
1,2008,9,8,10,0,293,17120,SB
2,2008,9,8,11,0,317,17120,SB


In [3]:
df['Datetime'] = pd.to_datetime(
    df['Yr'].astype(str) + '-' + 
    df['M'].astype(str).str.zfill(2) + '-' + 
    df['D'].astype(str).str.zfill(2) + ' ' + 
    df['HH'].astype(str).str.zfill(2) + ':' + 
    df['MM'].astype(str).str.zfill(2),
    format='%Y-%m-%d %H:%M'
)

df = df[df['Vol'] != -1]

direction_mapping = {'NB': 0, 'WB': 1, 'SB': 2, 'EB': 3}
df['Direction'] = df['Direction'].map(direction_mapping)

df.set_index(['Datetime', 'SegmentID', 'Direction'], inplace=True)
df.drop(['Yr', 'M', 'D', 'HH', 'MM'], axis=1, inplace=True)
df = df[~df.index.duplicated(keep='first')]
df.dropna(inplace=True)
df.reset_index(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79459 entries, 0 to 79458
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Datetime   79459 non-null  datetime64[ns]
 1   SegmentID  79459 non-null  int64         
 2   Direction  79459 non-null  int64         
 3   Vol        79459 non-null  int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 2.4 MB


In [4]:
df.set_index('Datetime', inplace=True)
df = (
    df.groupby(['SegmentID', 'Direction'])['Vol']
    .resample('15min')
    .mean()
    .interpolate()
    .reset_index()
)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285911 entries, 0 to 285910
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   SegmentID  285911 non-null  int64         
 1   Direction  285911 non-null  int64         
 2   Datetime   285911 non-null  datetime64[ns]
 3   Vol        285911 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 8.7 MB


In [5]:
df.head(3)

,SegmentID,Direction,Datetime,Vol
0,17120,2,2008-09-08 09:00:00,320.00
1,17120,2,2008-09-08 09:15:00,313.25
2,17120,2,2008-09-08 09:30:00,306.50


In [6]:
df.sort_values(by=['SegmentID', 'Direction', 'Datetime'], inplace=True)

df['Month'] = df['Datetime'].dt.month
df['Day'] = df['Datetime'].dt.day
df['Hour'] = df['Datetime'].dt.hour

df['Step'] = df['Datetime'].dt.minute // 15
df['Step'] = df['Step'].astype(int)

df['Direction'] = df['Direction'].astype(int)
df = df[['Datetime', 'Month', 'Day', 'Hour', 'Step', 'SegmentID', 'Direction', 'Vol']]
df.head(3)

,Datetime,Month,Day,Hour,Step,SegmentID,Direction,Vol
0,2008-09-08 09:00:00,9,8,9,0,17120,2,320.00
1,2008-09-08 09:15:00,9,8,9,1,17120,2,313.25
2,2008-09-08 09:30:00,9,8,9,2,17120,2,306.50


In [7]:
df.shape

(285911, 8)

In [8]:
df.describe()

,Datetime,Month,Day,Hour,Step,SegmentID,Direction,Vol
count,285911,285911.000000,285911.000000,285911.000000,285911.000000,285911.000000,285911.000000,285911.000000
mean,2007-12-26 20:44:24.114706944,5.865353,15.134538,11.504926,1.499687,98152.698983,1.611767,670.184049
min,2006-09-05 12:00:00,1.000000,1.000000,0.000000,0.000000,17120.000000,0.000000,0.000000
25%,2007-08-29 23:22:30,4.000000,8.000000,5.000000,0.000000,51124.000000,0.000000,122.000000
50%,2008-02-08 02:45:00,5.000000,14.000000,12.000000,1.000000,81472.000000,2.000000,283.000000
75%,2008-04-25 11:00:00,8.000000,22.000000,18.000000,2.000000,153405.000000,2.000000,566.000000
max,2008-09-16 09:00:00,12.000000,31.000000,23.000000,3.000000,194998.000000,3.000000,6184.000000
std,NaN,2.914918,8.571256,6.928529,1.118116,47323.080633,1.121901,945.754076


In [9]:
df['Datetime'] = pd.to_datetime(df['Datetime'])

# Check the data
print(df.head())

             Datetime  Month  Day  Hour  Step  SegmentID  Direction     Vol
0 2008-09-08 09:00:00      9    8     9     0      17120          2  320.00
1 2008-09-08 09:15:00      9    8     9     1      17120          2  313.25
2 2008-09-08 09:30:00      9    8     9     2      17120          2  306.50
3 2008-09-08 09:45:00      9    8     9     3      17120          2  299.75
4 2008-09-08 10:00:00      9    8    10     0      17120          2  293.00


In [10]:
import numpy as np

# Log transform (add 1 to avoid log(0))
df['Vol_log'] = np.log(df['Vol'] + 1)

In [11]:
# Extract more time-related features
df['Weekday'] = df['Datetime'].dt.dayofweek  # Monday=0, Sunday=6
df['Hour'] = df['Datetime'].dt.hour  # Hour of the day
df['Is_Weekend'] = df['Weekday'].isin([5, 6]).astype(int)  # Flag for weekends

In [12]:
# Check for missing values
print(df.isnull().sum())

# Fill missing values (forward fill for time series)
df['Vol'] = df['Vol'].fillna(method='ffill')

# Create a binary feature for zero traffic
df['Is_Traffic_Zero'] = (df['Vol'] == 0).astype(int)

Datetime      0
Month         0
Day           0
Hour          0
Step          0
SegmentID     0
Direction     0
Vol           0
Vol_log       0
Weekday       0
Is_Weekend    0
dtype: int64


C:\Users\vsing\AppData\Local\Temp\ipykernel_10412\2961015027.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Vol'] = df['Vol'].fillna(method='ffill')


In [13]:
# Create a differenced column for stationarity
df['Vol_diff'] = df['Vol'] - df['Vol'].shift(1)

# Drop rows with NaN after differencing
df.dropna(subset=['Vol_diff'], inplace=True)

In [14]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[['Vol', 'Vol_log', 'Vol_diff']] = scaler.fit_transform(df[['Vol', 'Vol_log', 'Vol_diff']])

In [15]:
import numpy as np

# Define a function to create sequences
def create_sequences(data, target, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length])
        y.append(target[i+sequence_length])
    return np.array(X), np.array(y)

# Choose features for the model
features = ['Vol', 'Vol_log', 'Vol_diff', 'Hour', 'Is_Weekend']
sequence_length = 97  # Example sequence length

# Create sequences
X, y = create_sequences(df[features].values, df['Vol'].values, sequence_length)

In [16]:
train_size = int(0.8 * len(X))
val_size = int(0.1 * len(X))  # 10% of the data for validation

# Split the data
X_train, X_temp = X[:train_size], X[train_size:]
y_train, y_temp = y[:train_size], y[train_size:]

# Further split the temporary set into validation and test sets
X_val, X_test = X_temp[:val_size], X_temp[val_size:]
y_val, y_test = y_temp[:val_size], y_temp[val_size:]

#### Creating LSTM

In [17]:
# Manually implementing sigmoid function
def sigmoid(x):
    return 1 / (1 + tf.exp(-x))

# Manually implementing tanh function
def tanh(x):
    return (tf.exp(x) - tf.exp(-x)) / (tf.exp(x) + tf.exp(-x))

In [18]:
# Manually implementing matrix multiplication
def matmul(x, y):
    return tf.matmul(x, y)

# Manually initializing weight matrices with Xavier (Glorot) uniform initialization
def glorot_uniform(shape):
    limit = np.sqrt(6.0 / (shape[0] + shape[1]))
    return tf.convert_to_tensor(np.random.uniform(low=-limit, high=limit, size=shape), dtype=tf.float32)

In [19]:
# Manually initializing bias vectors as zeros
def zeros_bias(shape):
    return tf.zeros(shape, dtype=tf.float32)

In [20]:
# Example usage of the above functions in your manual LSTM
def manual_lstm(inputs, units, return_sequences=False):
    time_steps = inputs.shape[1]
    features = inputs.shape[2]
    
    # Initialize weights for the LSTM gates (input, forget, output, and cell state)
    Wf = glorot_uniform((features, units))
    Uf = glorot_uniform((units, units))
    bf = zeros_bias((units,))
    
    Wi = glorot_uniform((features, units))
    Ui = glorot_uniform((units, units))
    bi = zeros_bias((units,))
    
    Wc = glorot_uniform((features, units))
    Uc = glorot_uniform((units, units))
    bc = zeros_bias((units,))
    
    Wo = glorot_uniform((features, units))
    Uo = glorot_uniform((units, units))
    bo = zeros_bias((units,))

    # Initialize hidden state and cell state for time t-1 (previous time step)
    h = tf.zeros((tf.shape(inputs)[0], units))  # Initial hidden state
    c = tf.zeros((tf.shape(inputs)[0], units))  # Initial cell state
    
    outputs = []

    for t in range(time_steps):
        x_t = inputs[:, t, :]  # Get input at time step t
        
        # Forget gate
        f_t = sigmoid(matmul(x_t, Wf) + matmul(h, Uf) + bf)
        
        # Input gate
        i_t = sigmoid(matmul(x_t, Wi) + matmul(h, Ui) + bi)
        
        # Candidate memory cell
        c_tilde_t = tanh(matmul(x_t, Wc) + matmul(h, Uc) + bc)
        
        # Output gate
        o_t = sigmoid(matmul(x_t, Wo) + matmul(h, Uo) + bo)
        
        # Update cell state
        c = f_t * c + i_t * c_tilde_t
        
        # Update hidden state
        h = o_t * tanh(c)
        
        outputs.append(h)  # Append the hidden state to the output list
    
    outputs = tf.stack(outputs, axis=1)  # Convert list of outputs to tensor

    if return_sequences:
        return outputs  # Return full sequence of hidden states
    else:
        return outputs[:, -1, :]  # Return the last hidden state only

In [21]:
def create_model(time_steps, features, units):
    # Input layer
    inputs = tf.keras.Input(shape=(time_steps, features))

    # Custom LSTM layer
    lstm_output = manual_lstm(inputs, units, return_sequences=False)

    # Dense layer for regression (no activation function for regression)
    dense_output = layers.Dense(1)(lstm_output)

    # Build the model
    model = tf.keras.Model(inputs, dense_output)
    
    return model

In [22]:
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [23]:
# Define model parameters
units = 64  # Number of units in LSTM
epochs = 10  # Number of epochs
batch_size = 32  # Batch size

# Create the model
model = create_model(sequence_length, len(features), units)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                    validation_data=(X_val, y_val), verbose=1)

Epoch 1/10


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

#check the test loss
test_loss = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss: {test_loss}")

# Predict traffic volumes on the test data
y_pred = model.predict(X_test)

#calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Mean Absolute Error: {mae}")
print(f"Root Mean Squared Error: {mse}")

In [ ]:
#plot training and validation loss and mae
plt.figure(figsize=(13, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')

plt.legend()
plt.show()

In [ ]:
y_test_with_dummy = np.column_stack((y_test, np.zeros_like(y_test), np.zeros_like(y_test)))
y_pred_with_dummy = np.column_stack((y_pred, np.zeros_like(y_pred), np.zeros_like(y_pred)))

# Inverse transform to revert to original scale (with all 3 columns)
y_test_original = scaler.inverse_transform(y_test_with_dummy)[:, 0]  # Extract the original 'Vol' column
y_pred_original = scaler.inverse_transform(y_pred_with_dummy)[:, 0]  # Extract the original 'Vol' column

ac_mae = mean_absolute_error(y_test_original, y_pred_original)
ac_rmse = np.sqrt(mean_squared_error(y_test_original, y_pred_original))

manualLSTM_accuracy = 100 * (1 - (ac_mae / np.mean(y_test_original)))

print(f"Mean Absolute Error (MAE): {ac_mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {ac_rmse:.2f}")
print(f"Accuracy: {manualLSTM_accuracy:.2f}%")

In [ ]:
# Plot the actual vs predicted traffic volumes
plt.figure(figsize=(7, 5))
plt.plot(y_test_original, color='blue', label='Actual Volume')
plt.plot(y_pred_original, color='red', label='Predicted Volume')
plt.xlabel('Actual Volume')
plt.ylabel('Predicted Volume')
plt.title('Actual vs Predicted Traffic Volume')
plt.show()

#### Comparing with inbuilt LSTM and MLP

In [ ]:
accuracies = [manualLSTM_accuracy]

In [ ]:
from tensorflow.keras.layers import LSTM

In [ ]:
#Training actual LSTM model
model = Sequential([
    LSTM(64, activation='tanh', input_shape=(sequence_length, len(features))),
    Dropout(0.2),  # Dropout to prevent overfitting
    Dense(1)  # Output layer to predict traffic volume
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=1  # Displays training progress
)

In [ ]:
#calculate test loss
test_loss = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss: {test_loss}")

# Predict traffic volumes on the test data
y_pred = model.predict(X_test)

#calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Mean Absolute Error: {mae}")
print(f"Root Mean Squared Error: {mse}")

In [ ]:
#invert transform to revert to original scale
y_test_with_dummy = np.column_stack((y_test, np.zeros_like(y_test), np.zeros_like(y_test)))
y_pred_with_dummy = np.column_stack((y_pred, np.zeros_like(y_pred), np.zeros_like(y_pred)))

y_test_original = scaler.inverse_transform(y_test_with_dummy)[:, 0]  # Extract the original 'Vol' column
y_pred_original = scaler.inverse_transform(y_pred_with_dummy)[:, 0]  # Extract the original 'Vol' column

ac_mae = mean_absolute_error(y_test_original, y_pred_original)
ac_rmse = np.sqrt(mean_squared_error(y_test_original, y_pred_original))

LSTM_accuracy = 100 * (1 - (ac_mae / np.mean(y_test_original)))

print(f"Mean Absolute Error (MAE): {ac_mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {ac_rmse:.2f}")
print(f"Accuracy: {LSTM_accuracy:.2f}%")

In [ ]:
accuracies.append(LSTM_accuracy)

#### Comparing Manual LSTM with inbuilt

In [ ]:
#plot a bar chart to compare the accuracies of the two models
plt.figure(figsize=(5, 5))
models = ['Manual LSTM', 'LSTM']
plt.bar(models, accuracies, color=['blue', 'red'], width=0.1)
plt.ylabel('Accuracy (%)')
plt.title('Manual LSTM vs LSTM')
plt.show()